이 자료는 위키독스 딥 러닝을 이용한 자연어 처리 입문의 FastText 튜토리얼 자료입니다.  

링크 : https://wikidocs.net/22883  

In [6]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [7]:
pip list | grep gensim

In [8]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

In [9]:
## 🌐 TED 강연 데이터(xml 파일)를 인터넷에서 다운로드하는 코드

import urllib.request   # 인터넷에서 파일을 다운로드할 수 있는 모듈

# urllib.request.urlretrieve("URL", filename="저장할_파일이름")
# → 지정한 URL에 있는 데이터를 내 컴퓨터(Colab 환경)에 저장

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml",
    filename="ted_en-20160408.xml"  # 다운로드 후 저장할 파일 이름
)

# ✅ 실행 결과:
# 현재 작업 폴더에 "ted_en-20160408.xml" 파일이 생성됩니다.
# (이 파일은 TED 영어 강연 자막 데이터셋으로, 텍스트 분류 실습에 자주 사용됩니다.)

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7fd874fac800>)

In [10]:
# 📘 TED 영어 강연 데이터(XML 파일)를 불러와서
#     텍스트만 추출하고, 문장/단어 단위로 토큰화하는 전체 전처리 과정

from lxml import etree        # XML 파일을 읽기 위한 라이브러리
import re                     # 정규 표현식(특정 패턴 문자 찾기/제거)
from nltk.tokenize import sent_tokenize, word_tokenize  # 문장/단어 단위 토큰화 도구

# ✅ 1️⃣ XML 파일 열기
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)   # XML 파일 구조로 파싱(읽어서 구조화)

# ✅ 2️⃣ <content>...</content> 사이의 내용만 추출
#   - TED 데이터에는 <content> 태그 안에 실제 강연 내용이 들어 있음
#   - xpath('//content/text()') → 모든 <content> 태그의 텍스트를 리스트로 가져옴
#   - '\n'.join(...) → 여러 문장을 줄바꿈(\n)으로 연결해 하나의 긴 문자열로 만듦
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# ✅ 3️⃣ (Audio), (Laughter) 등 괄호 안의 배경 설명 제거
#   - re.sub(패턴, 대체문자, 문자열)
#   - r'\([^)]*\)' → 괄호로 시작 '(' 후 ')' 전까지 모든 문자 제거
#   - 즉, ( )로 둘러싸인 모든 내용 삭제
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# ✅ 4️⃣ 문장 단위로 나누기 (Sentence Tokenization)
#   - sent_tokenize() : 문장을 마침표(.) 기준으로 분리
#   - 예: "Hello. I am a student." → ["Hello.", "I am a student."]
sent_text = sent_tokenize(content_text)

# ✅ 5️⃣ 각 문장별로 소문자 변환 + 구두점 제거
normalized_text = []
for string in sent_text:
    # re.sub(r"[^a-z0-9]+", " ", string.lower())
    # → 영어 알파벳(a~z)과 숫자(0~9)를 제외한 문자는 공백(" ")으로 치환
    # → 모든 대문자를 소문자로 바꿈 (lower())
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)
# 결과 예: "Hello, WORLD!!" → "hello world"

# ✅ 6️⃣ 단어 단위 토큰화 (Word Tokenization)
#   - 각 문장을 다시 단어별로 분리
#   - 예: ["hello world", "this is nlp"] → [["hello", "world"], ["this", "is", "nlp"]]
result = [word_tokenize(sentence) for sentence in normalized_text]

In [11]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [14]:
pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 59.3 MB/s eta 0:00:00


In [15]:
from gensim.models import Word2Vec, FastText

In [16]:
# 🧠 Word2Vec 모델 학습 (TED 영어 강연 데이터 기반)

from gensim.models import Word2Vec

# ✅ Word2Vec 모델 생성 및 학습
model = Word2Vec(
    sentences=result,   # 학습에 사용할 문장(단어 리스트들의 리스트)
    vector_size=100,    # 각 단어를 100차원 벡터로 표현 (단어 의미의 숫자 표현 크기)
    window=5,           # 중심 단어를 기준으로 앞뒤 5개의 단어를 문맥으로 사용
    min_count=5,        # 데이터 전체에서 5회 미만 등장한 단어는 무시
    workers=4,          # 동시에 학습할 CPU 코어 수 (병렬 처리 → 학습 속도 향상)
    sg=0                # 학습 방식: 0 = CBOW, 1 = Skip-gram
)

# ✅ 실행 결과:
# - model 안에 각 단어의 의미 벡터가 학습됨
# - model.wv['word'] 로 특정 단어의 벡터 확인 가능
# - model.wv.most_similar('word') 로 비슷한 의미의 단어 확인 가능

In [17]:
# 입력 단어에 대해서 유사한 단어를 찾아내는 코드에 electrofishing이라는 단어를 넣어봅니다.
model.wv.most_similar("electrofishing")

# 에러 메시지는 단어 집합(Vocabulary)에 electrofishing이 존재하지 않는다고 합니다.
# 이처럼 Word2Vec는 학습 데이터에 존재하지 않는 단어.
# 즉, 모르는 단어에 대해서는 임베딩 벡터가 존재하지 않기 때문에 단어의 유사도를 계산할 수 없습니다.

KeyError: "Key 'electrofishing' not present in vocabulary"

In [18]:
# ⚡ FastText 모델 학습 (TED 영어 강연 데이터 기반)
# 실행 시간은 약 3분 정도 소요될 수 있습니다.

from gensim.models import FastText

# ✅ FastText 모델 생성 및 학습
model = FastText(
    sentences=result,   # 학습용 데이터 (토큰화된 문장 리스트)
    vector_size=100,    # 각 단어를 100차원 벡터로 표현
    window=5,           # 중심 단어 기준으로 앞뒤 5단어를 문맥으로 사용
    min_count=5,        # 5회 미만 등장하는 단어는 무시
    workers=4,          # 학습에 사용할 CPU 코어 개수 (병렬 처리)
    sg=1                # 학습 알고리즘 선택 (1=Skip-gram, 0=CBOW)
)

# ✅ FastText란?
# - Word2Vec과 거의 비슷하지만, 단어를 “글자 단위(Subword)”까지 쪼개서 학습함
# - 예: “apple” → ["app", "ppl", "ple"] 형태의 n-gram 조각으로 학습
# - 따라서, 모르는 단어(OOV)도 구성 글자 정보를 이용해 벡터를 만들 수 있음

# ✅ 실행 결과:
# - model.wv['apple'] → "apple" 단어의 100차원 벡터
# - model.wv.most_similar('apple') → "apple"과 비슷한 의미의 단어 출력

In [19]:
# Word2Vec은 학습하지 않은 단어에 대해서 유사한 단어를 찾아내지 못했지만,
# FastText는 유사한 단어를 계산해서 출력하고 있음을 볼 수 있습니다.

model.wv.most_similar("electrofishing")

[('electrolyte', 0.8720206618309021),
 ('electrolux', 0.8692356944084167),
 ('electroshock', 0.8592448830604553),
 ('electro', 0.8582302927970886),
 ('electric', 0.8317796587944031),
 ('electron', 0.8255887031555176),
 ('electroencephalogram', 0.8247286081314087),
 ('electrochemical', 0.8222823143005371),
 ('electronic', 0.8212001323699951),
 ('electrogram', 0.8174077272415161)]